## Stationary test

We need to check for stationary before applying the Granger Causality test. Applying the test for non-stationary time series will overestimate the Granger Causality.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

## Import stock data

Apple, Google, Microsoft, NVidia, Amazon, Meta and Taiwan Semiconductor Manufacturing between January 1, 2023 and January 1, 2024.

In [5]:
from pandas_datareader import data as pdr

apple = yf.Ticker("AAPL")
google = yf.Ticker("GOOG")
microsoft = yf.Ticker("MSFT")
nvidia = yf.Ticker("NVDA")
amazon = yf.Ticker("AMZN")
meta = yf.Ticker("META")
tsmc = yf.Ticker("TSM")

aapl_data = apple.history(start='2023-01-01', end='2024-01-01')
goog_data = google.history(start='2023-01-01', end='2024-01-01')
msft_data = microsoft.history(start='2023-01-01', end='2024-01-01')
nvda_data = nvidia.history(start='2023-01-01', end='2024-01-01')
amzn_data = amazon.history(start='2023-01-01', end='2024-01-01')
meta_data = meta.history(start='2023-01-01', end='2024-01-01')
tsmc_data = tsmc.history(start='2023-01-01', end='2024-01-01')

In [6]:
datas = [aapl_data, goog_data, msft_data, nvda_data, amzn_data, meta_data, tsmc_data] 

## ADF Tests for stationary

Null hypothesis: Time series is not stationary \
Alternative hypothesis: If null hypothesis is rejected, then time series is stationaryry.

In [3]:
results_adf = pd.DataFrame(columns=['Company','Close','Close Difference','Daily Return'])
results_adf['Company'] = ['Apple','Google','Microsoft','NVIDIA','Amazon','Meta','TSMC']

In [7]:
## ADF tests on datas
for i, data in enumerate(datas):
    if adfuller(data['Close'].values)[1] < 0.05: # p value < 0.05:
        results_adf.loc[i,'Close'] = 'Stationary'
    else:
        results_adf.loc[i,'Close'] = 'Non-stationary'
    if adfuller(data['Close'].diff().dropna())[1] < 0.05: # p value < 0.05:
        results_adf.loc[i,'Close Difference'] = 'Stationary'
    else:
        results_adf.loc[i,'Close Difference'] = 'Non-stationary'
    if adfuller(data['Close'].pct_change().dropna())[1] < 0.05: # p value < 0.05:
        results_adf.loc[i,'Daily Return'] = 'Stationary'
    else:
       results_adf['Daily Return'][i] = 'Non-stationary'
print("ADF Test:")
results_adf

ADF Test:


,Company,Close,Close Difference,Daily Return
0,Apple,Non-stationary,Stationary,Stationary
1,Google,Non-stationary,Stationary,Stationary
2,Microsoft,Non-stationary,Stationary,Stationary
3,NVIDIA,Non-stationary,Stationary,Stationary
4,Amazon,Non-stationary,Stationary,Stationary
5,Meta,Non-stationary,Stationary,Stationary
6,TSMC,Non-stationary,Stationary,Stationary


## KPSS Tests for stationary
Null hypothesis: Time series is stationary around a deterministic trend (i.e. trend-stationary) \
Alternative hypothesis: Time series is a non-stationary unit root.

Note that KPSS Test determines if series is stationary around a mean or linear trend, or non stationary due to a unit root.

In [8]:
results_kpss = pd.DataFrame(columns=['Company','Close','Close Difference','Daily Return'])
results_kpss['Company'] = ['Apple','Google','Microsoft','NVIDIA','Amazon','Meta','TSMC']

In [10]:
## KPSS tests on datas

for i, data in enumerate(datas):
    if kpss(data['Close'].values)[1] >= 0.05: # p value < 0.05:
        results_kpss.loc[i,'Close'] = 'Stationary'
    else:
        results_kpss.loc[i,'Close'] = 'Non-stationary'
    if kpss(data['Close'].diff().dropna())[1] >= 0.05: # p value < 0.05:
        results_kpss.loc[i,'Close Difference'] = 'Stationary'
    else:
        results_kpss.loc[i,'Close Difference'] = 'Non-stationary'
    if kpss(data['Close'].pct_change().dropna())[1] >= 0.05: # p value < 0.05:
        results_kpss.loc[i,'Daily Return'] = 'Stationary'
    else:
       results_kpss['Daily Return'][i] = 'Non-stationary'
print("KPSS Test:")
results_kpss

KPSS Test:


C:\Users\teres\AppData\Local\Temp\ipykernel_11704\3693298409.py:4: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  if kpss(data['Close'].values)[1] >= 0.05: # p value < 0.05:
C:\Users\teres\AppData\Local\Temp\ipykernel_11704\3693298409.py:8: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  if kpss(data['Close'].diff().dropna())[1] >= 0.05: # p value < 0.05:
C:\Users\teres\AppData\Local\Temp\ipykernel_11704\3693298409.py:4: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  if kpss(data['Close'].values)[1] >= 0.05: # p value < 0.05:
C:\Users\teres\AppData\Local\Temp\ipykernel_11704\3693298409.py:8: InterpolationWarning: The test stati

,Company,Close,Close Difference,Daily Return
0,Apple,Non-stationary,Stationary,Stationary
1,Google,Non-stationary,Stationary,Stationary
2,Microsoft,Non-stationary,Stationary,Stationary
3,NVIDIA,Non-stationary,Stationary,Non-stationary
4,Amazon,Non-stationary,Stationary,Stationary
5,Meta,Non-stationary,Stationary,Stationary
6,TSMC,Non-stationary,Stationary,Stationary


## Why should we use both ADF and KPSS tests?
Because ADF tests for stationary, while KPSS tests for unit root. We will have the following 4 cases:

Case 1: Both ADF and KPSS tests conclude that the time series are stationary (just like our Daily Return). Then the series is stationary. \
Case 2: Both ADF and KPSS tests conclude that the time series are non-stationary (just like our Adj Close). Then the series is non-stationary. \
Case 3: KPSS indicates stationary while ADF indicates non-stationary. Then the series is trend stationary. To make the series strict stationary, remove the trend. The detrended series is checked for stationarity. \
Case 4: KPSS indicates non-stationary and ADF indicates stationary. Then the series is difference stationary. Differencing will make series stationary. The differenced series is checked for stationarity.

https://www.statsmodels.org/dev/examples/notebooks/generated/stationarity_detrending_adf_kpss.html?fbclid=IwZXh0bgNhZW0CMTAAAR03JaiW7WRnorhclQ4j-KvauL8RUnFcFctYIwgIuDIxjKoghJHFx-7JmYw_aem_AfsirxwMm_TAzO90SPYRO4LoqnojF6Xmb4zO2gsNLAxnzHx9AyXUKHwKLGOVYM5gViQQ5me4J9bmgbJi3iE6YXAj

We can apply Granger Causality on Apple's Close Difference and other Close Difference.